In [1]:
import numpy as np
import pandas as pd
import os
import sys

### Đọc dữ liệu

In [2]:
df = pd.read_csv("../data/onlret_transaction.csv", dtype={"CustomerID": str})

In [3]:
df.head(2)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Status,CustomerType
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,1,1
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,1,1


In [4]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'CustomerID', 'Country', 'TotalPrice', 'Status', 'CustomerType'],
      dtype='object')

In [5]:
df[df.InvoiceNo == "573585"].head(5)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Status,CustomerType
430237,573585,11001,2,2011-10-31 14:41:00,3.29,NaN,United Kingdom,6.58,1,0
430238,573585,15036,15,2011-10-31 14:41:00,1.63,NaN,United Kingdom,24.45,1,0
430239,573585,15039,15,2011-10-31 14:41:00,1.63,NaN,United Kingdom,24.45,1,0
430240,573585,15056BL,1,2011-10-31 14:41:00,12.46,NaN,United Kingdom,12.46,1,0
430241,573585,15056N,1,2011-10-31 14:41:00,12.46,NaN,United Kingdom,12.46,1,0


### Chứng minh nhận định

NỘI DUNG NHẬN ĐỊNH:

- Dữ liệu ban đầu **không là dạng chuỗi thời gian**, chỉ ở dạng transaction (giao dịch) thông thường.
- Khả năng chuyển đổi thành time-series: **Có**. Nếu xác định rõ đối tượng cần nghiên cứu (đối tượng phải tồn tại trong dữ liệu) thì sẽ biến đổi được về dạng time-series.


In [6]:
df.CustomerID.value_counts().head(7)

CustomerID
17841    7983
14911    5903
14096    5128
12748    4642
14606    2782
15311    2491
14646    2085
Name: count, dtype: int64

In [7]:
df[df.CustomerID == "14096"]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Status,CustomerType
317502,564764,16207B,1,2011-08-30 10:49:00,2.46,14096,United Kingdom,2.46,1,1
317503,564764,16225,1,2011-08-30 10:49:00,3.29,14096,United Kingdom,3.29,1,1
317504,564764,16254,1,2011-08-30 10:49:00,1.63,14096,United Kingdom,1.63,1,1
317505,564764,84249A,1,2011-08-30 10:49:00,0.83,14096,United Kingdom,0.83,1,1
317506,564764,84659A,2,2011-08-30 10:49:00,1.23,14096,United Kingdom,2.46,1,1
...,...,...,...,...,...,...,...,...,...,...
528083,580727,10135,8,2011-12-05 17:17:00,2.46,14096,United Kingdom,19.68,1,1
528084,580727,11001,3,2011-12-05 17:17:00,3.29,14096,United Kingdom,9.87,1,1
528085,580727,15044A,1,2011-12-05 17:17:00,5.79,14096,United Kingdom,5.79,1,1
528086,580727,15056BL,1,2011-12-05 17:17:00,12.46,14096,United Kingdom,12.46,1,1


### Chiến lược biến đổi thành data time-series

#### Chiến lược 1: Time-series ở mức hệ thống (System-level time series)

In [8]:
df["d"] = pd.to_datetime(df.InvoiceDate).dt.date

In [9]:
strategy_01 = df.groupby("d").agg(revenue = ("TotalPrice", "sum"), 
                    cnt_cus = ("CustomerID", "nunique"),
                    cnt_pro = ("StockCode", "nunique")
                   ).reset_index()

In [11]:
# đảm bảo cột d là datetime
strategy_01['d'] = pd.to_datetime(strategy_01['d'])

# tạo dải ngày đầy đủ từ min đến max
full_range = pd.date_range(
    start=strategy_01['d'].min(),
    end=strategy_01['d'].max(),
    freq='D'
)

# reindex theo dải ngày
strategy_01_full = (
    strategy_01
    .set_index('d')
    .reindex(full_range)
    .rename_axis('d')
    .reset_index()
)

# fill 0 cho các cột metric
cols_fill_zero = ['revenue', 'cnt_cus', 'cnt_pro']
strategy_01_full[cols_fill_zero] = strategy_01_full[cols_fill_zero].fillna(0)

In [12]:
strategy_01_full.to_csv("../data/strategy_01.csv", index=False)

In [22]:
strategy_01.head()

,d,revenue,cnt_cus,cnt_pro
0,2010-12-01,58635.56,98,1351
1,2010-12-02,46207.28,117,934
2,2010-12-03,45620.46,55,1156
3,2010-12-05,31383.95,76,1151
4,2010-12-06,53860.18,90,1479


#### Chiến lược 2: Time-series theo sản phẩm (Product-level time series)

In [20]:
strategy_02 = df.groupby(by=["d", "StockCode"]).agg(revenue = ("TotalPrice", "sum"), 
                    cnt_cus = ("CustomerID", "nunique"),
                    cnt_pro = ("Quantity", "sum")
                   ).reset_index()

In [22]:
# đảm bảo d là datetime
strategy_02['d'] = pd.to_datetime(strategy_02['d'])

# tạo calendar ngày đầy đủ
calendar = pd.DataFrame({
    'd': pd.date_range(
        start=strategy_02['d'].min(),
        end=strategy_02['d'].max(),
        freq='D'
    )
})

# merge để điền ngày thiếu
strategy_02_full = calendar.merge(
    strategy_02,
    on='d',
    how='left'
)

# fill StockCode
strategy_02_full['StockCode'] = strategy_02_full['StockCode'].fillna("")

# fill 0 cho các cột metric
cols_fill_zero = ['revenue', 'cnt_cus', 'cnt_pro']
strategy_02_full[cols_fill_zero] = strategy_02_full[cols_fill_zero].fillna(0)

In [23]:
strategy_02_full.to_csv("../data/strategy_02.csv", index=False)

In [24]:
strategy_02_full.shape, strategy_02.shape

((280520, 5), (280451, 5))

#### Chiến lược 3: Time-series theo khách hàng (Customer-level time series)

In [25]:
strategy_03 = df.groupby(by=["d", "CustomerID"]).agg(revenue = ("TotalPrice", "sum"), 
                    cnt_cus = ("StockCode", "nunique"),
                    cnt_pro = ("Quantity", "sum")
                   ).reset_index()

In [26]:
# đảm bảo d là datetime
strategy_03['d'] = pd.to_datetime(strategy_03['d'])

# tạo calendar ngày đầy đủ
calendar = pd.DataFrame({
    'd': pd.date_range(
        start=strategy_03['d'].min(),
        end=strategy_03['d'].max(),
        freq='D'
    )
})

# merge để điền ngày thiếu
strategy_03_full = calendar.merge(
    strategy_03,
    on='d',
    how='left'
)

# fill CustomerID
strategy_03_full['CustomerID'] = strategy_03_full['CustomerID'].fillna("")

# fill 0 cho các cột metric
cols_fill_zero = ['revenue', 'cnt_cus', 'cnt_pro']
strategy_03_full[cols_fill_zero] = strategy_03_full[cols_fill_zero].fillna(0)

In [27]:
strategy_03_full.to_csv("../data/strategy_03.csv", index=False)

In [28]:
strategy_03_full.shape, strategy_03.shape

((19365, 5), (19296, 5))

#### Chiến lược 4: Time-series đa biến (Multivariate time series)

In [14]:
strategy_04 = df.groupby("d").agg(revenue = ("TotalPrice", "sum"), 
                    cnt_cus = ("CustomerID", "nunique"),
                    cnt_pro = ("StockCode", "nunique"),
                    cnt = ("Quantity", "sum")      
                   ).reset_index()

In [16]:
# đảm bảo cột d là datetime
strategy_04['d'] = pd.to_datetime(strategy_04['d'])

# tạo dải ngày đầy đủ
full_range = pd.date_range(
    start=strategy_04['d'].min(),
    end=strategy_04['d'].max(),
    freq='D'
)

# reindex và điền ngày thiếu
strategy_04_full = (
    strategy_04
    .set_index('d')
    .reindex(full_range)
    .rename_axis('d')
    .reset_index()
)

# fill 0 cho các cột metric
cols_fill_zero = ['revenue', 'cnt_cus', 'cnt_pro', 'cnt']
strategy_04_full[cols_fill_zero] = strategy_04_full[cols_fill_zero].fillna(0)

In [17]:
strategy_04_full.to_csv("../data/strategy_04.csv", index=False)

In [18]:
strategy_04_full.shape

(374, 5)

In [19]:
strategy_04.shape

(305, 5)